# About this notebook
[home credit default risk competition by home credit](https://www.kaggle.com/competitions/home-credit-default-risk/data)'s [7th place notebook](https://www.kaggle.com/code/jsaguiar/lightgbm-7th-place-solution/script) is made of pile of long definition of functions. This notebook follows that. But in Korean

__another reference__ [홈크레딧 커널 스터디 by You Han Lee](https://www.youtube.com/watch?v=aoo1xrKQXFc&list=PLC_wC_PMBL5PexxXb6UC6pDXODE4xFWzL&index=3)

# default risk: 
디폴트는 채무 불이행, 파산이라는 뜻이죠. 파산이라고 하면 되겠습니다. 먼저 오류가 조금 있는데 7th place notebook이 아니라 7th place용으로 ensemble을 여러 개 썼는데 그 모듈 일부를 수정해서 올린 거라고 합니다. 또 이건 notebook이 아니라 스크립트에요.

따로 설명이 많지는 않습니다. [another similar version](https://github.com/js-aguiar/home-credit-default-competition) 이라는 링크가 있고, LightGBM의 boosting algorithm으로 goss를 썼다고 합니다. (다른 것으로는 gbdt가 있습니다) 데이터의 categorical feature를 label encode하는데 이게 앞의 구랑 무슨 관계인지 모르겠습니다.  

__핵심 아이디어__ 는 시간 관련 feature를 추가하는 거라고 합니다. 마지막 대출신청, 마지막 x개월 동안(대출신청의) 합계 등? 테이블간의 비율 뿐만 아니라 대출 종류의 취합? 조금 애매합니다.

## Import libraries
잔뜩 불러옵니다. 낯선 것들을 꼽아보자면
- gc : garbage collector
- contextlib.contextmanager : resource 관리를 위한 데코레이터 contextmanager를 쓰기 위해 불러온 라이브러리입니다. 이것은 timer를 정의하고 with timer 구문을 쓰기 위해 씁니다. 이것은 파이썬 표준 라이브러리에 포함돼 있습니다.
- multiprocessing : 표준 라이브러리입니다. mp.pool.Pool().map() 을 위해 씁니다.
- functools.partial
- scipy.stats.(kurtosis, iqr, skew)
- sklearn.metrics.roc_auc_score

In [80]:
import os
import gc
import time
import re
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# utility functions

## get age label

In [54]:
def get_age_label(days_birth):
    """ Return the age group label (int)"""
    age_years = -days_birth /365
    if age_years > 27: return 1
    elif age_years > 40: return 2
    elif age_years > 50: return 3
    elif age_years > 65: return 4
    elif age_years > 99: return 5
    else: return 0

## 평균, 중앙값, 표준편차

In [55]:
def do_mean(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
    columns={counted: agg_name})
    df=df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
    columns={counted: agg_name})
    df=df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
    columns={counted: agg_name})
    df=df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [56]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers by pandas.factorize"""
    if not categorical_columns:
        #if 'categorical_columns' is not given, take 'object' data type columns as categorical columns
        categorical_columns=[col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [57]:
def drop_application_columns(df):
    """drop features based on permutation feature importance"""
    drop_list=['CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE']
    #drop most flag document columns
    for doc_num in [2,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [58]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """one-hot encode categorical columns by pandas.get_dummies"""
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns=[col for col in df.columns if df[col].dtype=='object']
    df=pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns=[c for c in df.columns if c not in original_columns]
    return df, categorical_columns

In [59]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by=aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on=aggregate_by)


In [60]:
def group(df_to_agg, prefix, aggregations, aggregate_by = 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                              for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [61]:
def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp; gc.collect()
    return df


### add_features 는 아무데서도 쓰이지 않습니다.

```python
def add_features(feature_name, aggs, features, feature_names, groupby): #it is never called
    #feature_name is dictionary type
    feature_names.extend(['{}_{}'.format(feature_name, agg) for agg in aggs])
    
    for agg in aggs:
        #kurtosis and iqr are imported from scipy.stats
        if agg == 'kurt':
        elif agg == 'iqr':
            agg_func = iqr
        else:
            agg_func = agg
        
            agg_func = kurtosis 
        g = groupby[feature_name].agg(agg_func).
```

In [ ]:
def add_features_in_group(features, gr_, feature_name, aggs, prefix):
    """
    add_features_in_group(features, gr_, feature_name, aggs, prefix)
    add some features to the 'dictionary', applying functions on column of dataframe.
    features: dictionary
    gr_: (conditionally filtered) dataframe
    feature_name, prefix : string
    aggs: list of string
    """
    for agg in aggs:
        if agg == 'sum':
            features['{}{}_sum'.format(prefix, feature_name)] = gr_[feature_name].sum()
        elif agg == 'mean':
            features['{}{}_mean'.format(prefix, feature_name)] = gr_[feature_name].mean()
        elif agg == 'max':
            features['{}{}_max'.format(prefix, feature_name)] = gr_[feature_name].max()
        elif agg == 'min':
            features['{}{}_min'.format(prefix, feature_name)] = gr_[feature_name].min()
        elif agg == 'std':
            features['{}{}_std'.format(prefix, feature_name)] = gr_[feature_name].std()
        elif agg == 'count':
            features['{}{}_count'.format(prefix, feature_name)] = gr_[feature_name].count()
        elif agg == 'skew':
            features['{}{}_skew'.format(prefix, feature_name)] = skew(gr_[feature_name])
        elif agg == 'kurt':
            features['{}{}_kurt'.format(prefix, feature_name)] = kurt(gr_[feature_name])
        elif agg == 'iqr':
            features['{}{}_iqr'.format(prefix, feature_name)] = iqr(gr_[feature_name])
        elif agg == 'median':
            features['{}{}_median'.format(prefix, feature_name)] = gr_[feature_name].median()
    return features




In [ ]:
def add_trend_feature(features, gr, feature_name, prefix):
    """
    add_trend_feature(features, gr, feature_name, prefix)
    try linear regression on gr[feature_name] and add trend(gradient) to the dictionary and return,
        where x axis is [[0],[1],[2]....]
    features: dictionary
    gr: conditionally filltered dataframe
    """
    y = gr[feature_name].values
    try:
        x = np.arange(0, len(y)).reshape(-1, 1)
        lr = LinearRegression()
        lr.fit(x, y)
        trend = lr.coef_[0]
    except:
        trend = np.nan
    features['{}{}'.format(prefix, feature_name)] = trend
    return features

In [ ]:
def parallel_apply(groups, func, index_name='Index', num_workers=0, chunk_size=100_000):
    """ 
    multiprocessing.pool.Pool().map() is parallel equivalent of map() (built in function).
    see
    https://docs.python.org/ko/3/library/multiprocessing.html#multiprocessing.pool.Pool
    for details."""
    if num_workers <= 0: num_workers = NUM_THREADS
    #n_chunks = np.ceil(1.0 * groups.ngroups/chunk_size) #?
    indices, features = [], []
    for index_chunk, group_chunk in chunk_groups(groups, chunk_size):
        with mp.pool.Pool(num_workers) as executor:
            features_chunk = executor.map(func, group_chunk)
        features.extend(features_chunk)
        indeces.extend(index_chunk)
    
    features = pd.DataFrame(features)
    features.index = indeces
    features.index.name = index_name
    return features
        
def chunk_groups(groupby_object, chunk_size):
    """yield sub-dataframe divided by groupby."""
    n_groups = groupby_object.ngroups
    group_chunk, index_chunk = [], []
    for i, (index, df) in enumerate(groupby_object):
        group_chunk.append(df)
        group_index.append(index)
        if (i+1) % chunk_size == 0 or i + 1 ==n_groups:
            group_chunk_, index_chunk_ = group_chunk.copy(), index_chunk.copy()
            group_chunk, index_chunk = [], []
            yield index_chunk_, group_chunk_

In [ ]:
def reduce_memory(df):
    """reduce memory usage of a dataframe by setting data types"""
    start_mem = df.memory_usage().sum() / 1024 **2
    #1024**2 byte = 1 mega byte
    print("Initial df memory usage: {:.2f} MB for {} columns"
          .format(start_mem, len(df.columns)))
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type[:3])== 'int':
                #can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax<np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax<np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax<np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax<np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax<np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax<np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                df[col] = df[col].astype(np.float64)
    end_mem =df.memory_usage().sum() / 1024**2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is {:.2f} MB - {:.1f}% decreased'.format(end_mem, memory_reduction))
    return df


# 구문 해석
하나하나 살펴봅시다. 먼저 실행되는 순서를 살펴봅니다. 함수 정의를 다 건너뛰고 보면 'configuration'이 나옵니다. 이것 역시 변수의 할당일 뿐 실행은 아닙니다. 

그러면 1083-1087행이 남습니다.

```python
#1083 - 1087 rows
if __name__ == "__main__":
    pd.set_option('display.max_rows', 60)
    pd.set_option('display.max_columns', 100)
    with timer('Pipeline total time"):
              main(debug = False)
```
먼저 \_\_name\_\_ 은 사용자가 실행시킨 스크립트(.py)파일에서 직접 호출되면 \_\_main\_\_ 이란 값을, 그 외의 모듈에서 실행되면 모듈의 이름을 돌려주는 사전 정의된 변수라고 합니다. 밑줄 두개로 둘러쌓인 built in method들을 magic method라고 하고, 개중에는 \_\_init\_\_ 이게 제일 익숙할 것 같습니다.
호출순서대로는 timer가 먼저 등장합니다.

## timer
이 함수의 정의를 살펴보면 contextmanager 데코레이터가 붙어있습니다. with a as b:, yield 이랑 같이 가는 거라고 보면 되겠습니다. 자세한 설명은 [python 공식문서](https://docs.python.org/ko/3/library/contextlib.html) 나 [도장.io](https://dojang.io/mod/page/view.php?id=2412) 를 참고하는게 좋겠습니다. 도장io 사이트는 그냥 익숙한 사이트라서 봅니다.




In [62]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(name, time.time()-t0))

## main function
그 다음으로는 main(debug=False)로 main function을 실행하는데 이걸 봐야겠네요.

In [36]:
def main(debug = False):
    num_rows = 30000 if debug else None
    #load data, merge, and clean memory
    with timer("application_train and application_test"):
        df = get_train_test(DATA_DIRECTORY, num_rows=num_rows)
        print("Application dataframe shape: ", df.shape)
    with timer("Bureau and bureau_balancce data"):
        bureau_df=get_bureau(DATA_DIRECTORY, num_rows=num_rows)
        df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')

        print("Bureau dataframe shape: ", bureau_df.shape)
        del bureau_df; gc.collect()
    with timer("previous_application"):
        prev_df = get_previous_applications(DATA_DIRECTORY, num_rows)
        df=pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
        del prev_df; gc.collect()
    with timer("previous applications balance"):
        pos=get_pos_cash(DATA_DIRECTORY, num_rows)
        df=pd.merge(df, pos, on='SK_ID_CURR', how='left')
        print("Pos-cash dataframe shape: ", pos.shape)
        del pos; gc.collect()
        ins = get_installment_payments(DATA_DIRECTORY, num_rows)
        df=pd.merge(df, ins, on='SK_ID_CURR', how='left')
        print("Installments dataframe shape: ", ins.shape)
        del ins; gc.collect()
        cc=get_credit_card(DATA_DIRECTORY, num_rows)
        df=pd.merge(df, cc, on='SK_ID_CURR', how='left')
        print("Credit card dataframe shape: ", cc.shape)
        del cc; gc.collect()
    #add ratios and groupby between different tables
    df=add_ratios_features(df)
    df=reduce_memory(df)
    lgbm_categorical_feat=[
        'CODE_GENDER', 'FLAG_OWN_CAR','NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
        'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
        'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']
    with timer("Run LightGBM"):
        feat_importance = kfold_lightgbm_sklearn(df, lgbm_categorical_feat)
        print(feat_importance)

In [63]:
def add_ratios_features(df):
    #credit to income ratio:
    df['BUREAU_INCOME_CREDIT_RATIO']=df['BUREAU_AMT_CREDIT_SUM_MEAN'] / df['AMT_INCOME_TOTAL']
    df['BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO']=df['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM'] / df['AMT_INCOME_TOTAL']
    #previous to current credit ratio
    df['CURRENT_TO_APPROVED_CREDIT_MIN_RATIO']=df['APPROVED_AMT_CREDIT_MIN']/df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MAX_RATIO']=df['APPROVED_AMT_CREDIT_MAX']/df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MEAN_RATIO']=df['APPROVED_AMT_CREDIT_MEAN']/df['AMT_CREDIT']
    #PREVIOUS TO CURRENT ANNUITY RATIO
    df['CURRENT_TO_APPROVED_ANNUITY_MAX_RATIO']=df['APPROVED_AMT_ANNUITY_MAX']/df['AMT_ANNUITY']
    df['CURREMT_TO_APPROVED_ANNUITY_MEAN_RATIO']=df['APPROVED_AMT_ANNUITY_MEAN']/df['AMT_ANNUITY']
    df['PAYMENT_MIN_TO_ANNUITY_RATIO']=df['INS_AMT_PAYMENT_MIN']/df['AMT_ANNUITY']
    df['PAYMENT_MAX_TO_ANNUITY_RATIO']=df['INS_AMT_PAYMENT_MAX']/df['AMT_ANNUITY']
    df['PAYMENT_MEAN_TO_ANNUITY_RATIO']=df['INS_AMT_PAYMENT_MEAN']/df['AMT_ANNUITY']
    #PREVIOUS TO CURRENT CREDIT TO ANNUITY RATIO
    df['CTA_CREDIT_TO_ANNUITY_MAX_RATIO']=df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MAX']/df['CREDIT_TO_ANNUITY_RATIO']
    df['CTA_CREDIT_TO_ANNUITY_MEAN_RATIO']=df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MEAN']/df['CREDIT_TO_ANNUITY_RATIO']
    #DAYS DIFFERENCES AND RATIOS
    df['DAYS_DECISION_MEAN_TO_BIRTH']=df['APPROVED_DAYS_DECISION_MEAN'] / df['DAYS_BIRTH']
    df['DAYS_CREDIT_MEAN_TO_BIRTH']=df['BUREAU_DAYS_CREDIT_MEAN'] / df['DAYS_BIRTH']
    df['DAYS_DECISION_MEAN_TO_EMPLOYED']=df['APPROVED_DAYS_DECISION_MEAN'] / df['DAYS_EMPLOYED']
    df['DAYS_CREDIT_MEAN_TO_EMPLOYED']=df['BUREAU_DAYS_CREDIT_MEAN']/df['DAYS_EMPLOYED']
    return df
    

# LightGBM Model

In [89]:
def kfold_lightgbm_sklearn(data, categorical_feature = None):
    data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_\s]+', '', x)) #added line to fix character error

    df=data[data['TARGET'].notnull()]
    test= data[data['TARGET'].isnull()]
    print("Train/valid shape: {}, test_shape: {}".format(df.shape, test.shape))
    del_features = ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index', 'level_0']
    predictors = list(filter(lambda v: v not in del_features, df.columns))

    
    
    if not STRATIFIED_KFOLD:
        folds=KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    else:
        folds=StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    
    #Hold oof predictions, test predictions, feature importance and training/valid auc
    oof_preds = np.zeros(df.shape[0])
    sub_preds = np.zeros(test.shape[0])
    importance_df=pd.DataFrame()
    eval_results=dict()
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df[predictors], df['TARGET'])):
        train_x, train_y = df[predictors].iloc[train_idx], df['TARGET'].iloc[train_idx]
        valid_x, valid_y = df[predictors].iloc[valid_idx], df['TARGET'].iloc[valid_idx]
        
        params = {'random_state': RANDOM_SEED, 'nthread': NUM_THREADS}
        clf = LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
        if not categorical_feature:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                   eval_metric='auc', verbose=400, early_stopping_rounds=EARLY_STOPPING)
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
                   eval_metric='auc', verbose=400, early_stopping_rounds=EARLY_STOPPING,
                   feature_name=list(df[predictors].columns), categorical_feature=categorical_feature)
        oof_preds[valid_idx]=clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:,1]
        sub_preds += clf.predict_proba(test[predictors], num_iteration=clf.best_iteration_)[:,1]/folds.n_splits
        
        #feature importance by gain and split
        fold_importance = pd.DataFrame()
        fold_importance["feature"]=predictors
        fold_importance["gain"]=clf.booster_.feature_importance(importance_type='gain')
        fold_importance["split"]=clf.booster_.feature_importance(importance_type='split')
        importance_df=pd.concat([importance_df, fold_importance], axis=0)
        eval_results['train_{}'.format(n_fold+1)] = clf.evals_result_['training']['auc']
        eval_results['valid_{}'.format(n_fold+1)] = clf.evals_result_['valid_1']['auc']
        
        print('FOLD %2d AUC : %.6f' % (n_fold+1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(df['TARGET'], oof_preds))
    test['TARGET']=sub_preds.copy()
    
    #get the average feature importance between folds
    mean_importance = importance_df.groupby('feature').mean().reset_index()
    mean_importance.sort_values(by='gain', ascending=False, inplace=True)
    #save feature importance, test predictions and oof predictions as csv
    if GENERATE_SUBMISSION_FILES:
        
        #generative oof csv
        oof=pd.DataFrame()
        oof['SK_ID_CURR']=df['SK_ID_CURR'].copy()
        df['PREDICTION']=oof_preds.copy()
        df['TARGET']=df['TARGET'].copy()
        df.to_csv('oof{}.csv'.format(SUBMISSION_SUFIX), index=False)
        #save submission (test data) and feature importance
        test[['SK_ID_CURR', 'TARGET']].to_csv('submission{}.csv'.format(SUBMISSION_SUFIX), index=False)
        mean_importance.to_csv('feature_importance{}.csv'.format(SUBMISSION_SUFIX), index=False)
    return mean_importance
        

# get_train_test

In [65]:
def get_train_test(path, num_rows = None):
    """process application_train.csv and application_test.csv and return a pandas dataframe"""
    train = pd.read_csv(os.path.join(path, 'application_train.csv'), nrows=num_rows)
    test = pd.read_csv(os.path.join(path, 'application_test.csv'), nrows=num_rows)
    df=train.append(test)
    del train, test; gc.collect()
    #data cleaning
    df = df[df['CODE_GENDER'] != 'XNA'] #4 people with XNA code gender
    df = df[df['AMT_INCOME_TOTAL']<20_000_000]
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
    
    #Flag_document features - count and kurtosis
    docs = [f for f in df.columns if 'FLAG_DOC' in f]
    df['DOCUMENT_COUNT']=df[docs].sum(axis=1)
    df['NEW_DOC_KURT']=df[docs].kurtosis(axis=1)
    #categorical age - based on target=1 plot
    df['AGE_RANGE']=df['DAYS_BIRTH'].apply(get_age_label)
    
    #new features based on External sources
    df['EXT_SOURCES_PROD']=df['EXT_SOURCE_1']*df['EXT_SOURCE_2']*df['EXT_SOURCE_3']
    df['EXT_SOURCES_WEIGHTED']=df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 + df.EXT_SOURCE_3 * 3
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name]=eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)
    
    #credit ratios
    df['CREDIT_TO_ANNUITY_RATIO']=df['AMT_CREDIT']/df['AMT_ANNUITY']
    df['CREDIT_TO_GOODS_RATIO']=df['AMT_CREDIT']/df['AMT_GOODS_PRICE']
    #income ratios
    df['ANNUITY_TO_INCOME_RATIO']=df['AMT_ANNUITY']/df['AMT_INCOME_TOTAL']
    df['CREDIT_TO_INCOME_RATIO']=df['AMT_CREDIT']/df['AMT_INCOME_TOTAL']
    df['INCOME_EMPLOYED_RATIO']=df['AMT_INCOME_TOTAL']/df['DAYS_EMPLOYED']
    df['PHONE_TO_BIRTH_RATIO']=df['AMT_INCOME_TOTAL']/df['DAYS_BIRTH']
    #time_ratios
    df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED']/df['DAYS_BIRTH']
    df['ID_TO_BIRTH_RATIO']=df['DAYS_ID_PUBLISH']/df['DAYS_BIRTH']
    df['CAR_TO_BIRTH_RATIO']=df['OWN_CAR_AGE']/df['DAYS_BIRTH']
    df['CAR_TO_EMPLOYED_RATIO']=df['OWN_CAR_AGE']/df['DAYS_EMPLOYED']
    df['PHONE_TO_BIRTH_RATIO']=df['DAYS_LAST_PHONE_CHANGE']/df['DAYS_BIRTH']

    #GroupyBy : Statistics for applications in the same group
    group=['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
    df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
    
    #below is copied- will be deleted-- 
    # Groupby: Statistics for applications in the same group
    group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
    df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
    df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
    df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
    df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
    df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
    df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
    df = do_mean(df, group, 'AMT_CREDIT', 'GROUP_CREDIT_MEAN')
    df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
    df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

    # Encode categorical features (LabelEncoder)
    df, le_encoded_cols = label_encoder(df, None)
    df = drop_application_columns(df)
    return df

# GET_BUREAU
BUREAU라는 다른 회사 데이터.

In [66]:
def get_bureau(path, num_rows=None):
    """load bureau.csv and bureau_balance.csv, process, return pandas dataframe"""
    bureau=pd.read_csv(os.path.join(path, 'bureau.csv'), nrows=num_rows)
    #credit duration and credit/account end date difference
    bureau['CREDIT_DURATION']=-bureau['DAYS_CREDIT']+bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF']=bureau['DAYS_CREDIT_ENDDATE']-bureau['DAYS_ENDDATE_FACT']
    #credit to debt ratio and difference
    bureau['DEBT_PERCENTAGE']=bureau['AMT_CREDIT_SUM']/bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF']=bureau['AMT_CREDIT_SUM']-bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO']=bureau['AMT_CREDIT_SUM']/bureau['AMT_ANNUITY']
    
    #one-hot encoder
    bureau, categorical_cols=one_hot_encoder(bureau, nan_as_category=False)
    #join bureau balance features
    bureau = bureau.merge(get_bureau_balance(path, num_rows), how='left', on='SK_ID_BUREAU')
    
    #Flag months with late payments (days past due)
    bureau['STATUS_12345']=0
    for i in range(1,6):
        bureau['STATUS_12345']+=bureau['STATUS_{}'.format(i)]
    
    # aggregate by number of months in balance and merge with bureau (loan length agg)
    features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
        'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']
    agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean().reset_index()
    agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)
    bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')
    del agg_length; gc.collect()
    
    #General loans aggregations
    agg_bureau = group(bureau, 'BUREAU_', BUREAU_AGG)
    #Active and closed loans aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    agg_bureau = group_and_merge(active, agg_bureau, 'BUREAU_ACTIVE_', BUREAU_ACTIVE_AGG)
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    agg_bureau = group_and_merge(closed, agg_bureau, 'BUREAU_CLOSED_', BUREAU_CLOSED_AGG)
    del active, closed; gc.collect()
    # Aggregations for the main loan types
    for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
        type_df = bureau[bureau['CREDIT_TYPE_' + credit_type] == 1]
        prefix = 'BUREAU_' + credit_type.split(' ')[0].upper() + '_'
        agg_bureau = group_and_merge(type_df, agg_bureau, prefix, BUREAU_LOAN_TYPE_AGG)
        del type_df; gc.collect()
    #Time based aggregations
    for time_frame in [6, 12]:
        prefix = 'BUREAU_LAST{}M_'.format(time_frame)
        time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -30*time_frame]
        agg_bureau = group_and_merge(time_frame_df, agg_bureau, prefix, BUREAU_TIME_AGG)
        del time_frame_df; gc.collect()
    
    # Last loan max overdue
    sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
    gr = sort_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
    gr.rename({'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
    agg_bureau = agg_bureau.merge(gr, on='SK_ID_CURR', how='left')
    # total debt/total credit and active loans debt / active loans credit
    agg_bureau['BUREAU_DEPT_OVER_CREDIT'] = \
        agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
    agg_bureau['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = \
        agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']
    return agg_bureau

## GET_BUREAU_BALANCE

In [67]:
def get_bureau_balance(path, num_rows=None):
    bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows=num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category = False)
    # rate for each category with decay?
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # min, max, count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE':['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

In [42]:
#def get_previous_applications()

def get_previous_applications(path, num_rows = None):
    """Process previous_application.csv and return a pandas DataFrame"""
    prev=pd.read_csv(os.path.join(path, 'previous_application.csv'), nrows=num_rows)
    pay=pd.read_csv(os.path.join(path, 'installments_payments.csv'), nrows=num_rows)
    
    # one - hot encode most important categorical features
    ohe_columns=[
        'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'CHANNEL_TYPE',
        'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
        'NAME_PRODUCT_TYPE', 'NAME_CLIENT_TYPE']
    prev, categorical_cols = one_hot_encoder(prev, ohe_columns, nan_as_category = False)
    
    #add Features: ratios and differences
    prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['APPLICATION_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT']/prev['AMT_ANNUITY']
    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']
    #interest ratio on previous application (simplified)
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (total_payment/prev['AMT_CREDIT'] - 1)/prev['CNT_PAYMENT']
    
    #Active loans: approved, not complete
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    active_df = approved[approved['DAYS_LAST_DUE'] == 365243]
    
    #Find how much was aleardy payed in active loans (using installments csv)
    active_pay = pay[pay['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]
    active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
    active_pay_agg.reset_index(inplace = True)
    #active loans: difference between payed and installments
    active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']
    #merge with active_df
    active_df = active_df.merge(active_pay_agg, on= 'SK_ID_PREV', how='left')
    active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - active_df['AMT_PAYMENT']
    active_df['REPAYMENT_RATIO'] = active_df['AMT_PAYMENT'] / active_df['AMT_CREDIT']
    #perform aggregations for active applications
    active_agg_df = group(active_df, 'PREV_ACTIVE_', PREVIOUS_ACTIVE_AGG)
    active_agg_df(['TOTAL_REPAYMENT_RATIO'] = active_agg_df['PREV_ACTIVE_AMT_PAYMENT_SUM']/\
                 active_agg_df['PREV_ACTIVE_AMT_CREDIT_SUM'])
    del active_pay, active_pay_agg, active_df; gc.collect()
                  
    #change 365243 to nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    #Days last due difference (scheduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']
    
    #categorical features
    categorical_agg = {key: ['mean'] for key in categorical_cols}
    #perform general aggregations
    agg_prev = group(prev, 'PREV_', {**PREVIOUS_AGG, **categorical_agg})
    #Merge active loans df on agg_prev
    agg_prev = agg_prev.merge(active_agg_df, how='left', on='SK_ID_CURR')
    del active_agg_df; gc.collect()
    #aggregations for approved and refused loans
    agg=prev = group_and_merge(approved, agg_prev, 'APPROVED_', PREVIOUS_APPROVED_AGG)
    refuesd = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    agg_prev = group_and_merge(refused, agg_prev, 'REFUSED_', PREVIOUS_REFUSED_AGG)
    del approved, refused; gc.collect()
    #aggregations for consumer loans and cash loans
    for loan_type in ['Consumer loans', 'Cash loans']:
        type_df = prev[prev['NAME_CONTRACT_TYPE_{}'.format(loan_type)] == 1]
        prefix = 'PREV_' + loan_type.split(" ")[0] + '_'
        agg_prev = group_and_merge(type_df, agg_prev, prefix, PREVIOUS_LOAN_TYPE_AGG)
        del type_df; gc.collect()
    
    #get the SK_ID_PREV for loans with late payments (days past due: DPD)
    pay['LATE_PAYMENT'] = pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']
    pay['LATE_PAYMENT'] = pay['LATE_PAYMENT'].apply(lambda x: 1 if x > 0 else 0)
    dpd_id = pay[pay['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()
    #Aggregations for loans with late payment
    agg_dpd=group_and_merge(prev[prev['SK_ID_PREV'].isin(dpd_id)], agg_prev,
                            'PREV_LATE_', PREVIOUS_LATE_PAYMENTS_AGG)
    del agg_dpd, dpd_id; gc.collect()
    #Aggregations for loans in the last x months
    for time_frame in [12, 24]:
        time_frame_df = prev[prev['DAYS_DECISION'] >= -30*time_frame]
        prefix = 'PREV_LAST{}M_'.format(time_frame)
        agg_prev = group_and_merge(time_frame_df, agg_prev, prefix, PREVIOUS_TIME_AGG)
        del time_frame_df; gc.collect()
    del prev; gc.collect()
    return agg_prev



# GET_POS_CASH... etc

In [68]:
def get_pos_cash(path, num_rows = None):
    """Process POS_CASH_balance.csv and return pandas dataframe"""
    pos = pd.read_csv(os.path.join(path, 'POS_CASH_balance.csv'), nrows = num_rows)
    pos, categorical_cols = one_hot_encoder(pos, nan_as_category=False)
    #Flag months with late payment
    pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    #aggregate by SK_ID_CURR
    categorical_agg = {key: ['mean'] for key in categorical_cols}
    pos_agg = group(pos, 'POS_', {**POS_CASH_AGG, *categorical_agg})
    #sort and group by SK_ID_PREV
    sort_pos = pos.sort_values(by = ['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.groupby('SK_ID_PREV')
    df = pd.DataFrame()
    df['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
    df['MONTHS_BALANCE_MAXX'] = gp['MONTHS_BALANCE'].max()
    #percentage of previous loans completed and completed before initial term
    df['POS_LOAN_COMPLTED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
    df['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INTALMENT'].first() - gp['CNT_INSTALMENT'].last()
    df['POS_COMPLETED_BEFORE_MENN'] = df.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0
                                              and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)
    #number of remaining installments (future installments) and percentage from total
    df['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALLMENT_FUTURE'].last()
    df['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()
    #group by SK_ID_CURR and merge
    df_gp = df.groupby('SK_ID_CURR').sum().reset_index()
    df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace=True)
    pos_agg = pd.merge(pos_agg, df_gp, on='SK_ID_CURR', how='left')
    def df, gp, df_gp, sort_pos; gc.collect()
    
    #percentage of late payments for the 3 most recent applications
    pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')
    #last month of each application
    last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    #most recent applications (last 3)
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
    gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
    pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')
    # drop some usless categories: too small. does not appear if you set num_rows=30000
    drop_features = ['POS_NAME_CONTRACT_STATUS_Canceled_MEAN', 'POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN', 'POS_NAME_CONTRACT_STATUS_XNA_MEAN']
    for drop_feature in drop_features:
        try:
            pos_agg.drop(drop_feature, axis=1, inplace=True)
        except: pass
    return pos_agg
    
    


In [ ]:
#def get_installment_payments(path, num_rows = None):
#    """Process installments_payments.csv and return a pandas dataframe"""
#    pay = pd.read_csv(os.path.join(path, 'installments_payments.csv'), nrows=num_rows)
#group payments and get payment difference
    
# below are copied

def get_installment_payments(path, num_rows= None):
    """ Process installments_payments.csv and return a pandas dataframe. """
    pay = pd.read_csv(os.path.join(path, 'installments_payments.csv'), nrows= num_rows)
    # Group payments and get Payment difference
    pay = do_sum(pay, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')
    pay['PAYMENT_DIFFERENCE'] = pay['AMT_INSTALMENT'] - pay['AMT_PAYMENT_GROUPED']
    pay['PAYMENT_RATIO'] = pay['AMT_INSTALMENT'] / pay['AMT_PAYMENT_GROUPED']
    pay['PAID_OVER_AMOUNT'] = pay['AMT_PAYMENT'] - pay['AMT_INSTALMENT']
    pay['PAID_OVER'] = (pay['PAID_OVER_AMOUNT'] > 0).astype(int)
    # Payment Entry: Days past due and Days before due
    pay['DPD'] = pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']
    pay['DPD'] = pay['DPD'].apply(lambda x: 0 if x <= 0 else x)
    pay['DBD'] = pay['DAYS_INSTALMENT'] - pay['DAYS_ENTRY_PAYMENT']
    pay['DBD'] = pay['DBD'].apply(lambda x: 0 if x <= 0 else x)
    # Flag late payment
    pay['LATE_PAYMENT'] = pay['DBD'].apply(lambda x: 1 if x > 0 else 0)
    # Percentage of payments that were late
    pay['INSTALMENT_PAYMENT_RATIO'] = pay['AMT_PAYMENT'] / pay['AMT_INSTALMENT']
    pay['LATE_PAYMENT_RATIO'] = pay.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)
    # Flag late payments that have a significant amount
    pay['SIGNIFICANT_LATE_PAYMENT'] = pay['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
    # Flag k threshold late payments
    pay['DPD_7'] = pay['DPD'].apply(lambda x: 1 if x >= 7 else 0)
    pay['DPD_15'] = pay['DPD'].apply(lambda x: 1 if x >= 15 else 0)
    # Aggregations by SK_ID_CURR
    pay_agg = group(pay, 'INS_', INSTALLMENTS_AGG)

    # Installments in the last x months
    for months in [36, 60]:
        recent_prev_id = pay[pay['DAYS_INSTALMENT'] >= -30*months]['SK_ID_PREV'].unique()
        pay_recent = pay[pay['SK_ID_PREV'].isin(recent_prev_id)]
        prefix = 'INS_{}M_'.format(months)
        pay_agg = group_and_merge(pay_recent, pay_agg, prefix, INSTALLMENTS_TIME_AGG)

    # Last x periods trend features
    group_features = ['SK_ID_CURR', 'SK_ID_PREV', 'DPD', 'LATE_PAYMENT',
                      'PAID_OVER_AMOUNT', 'PAID_OVER', 'DAYS_INSTALMENT']
    gp = pay[group_features].groupby('SK_ID_CURR')
    func = partial(trend_in_last_k_instalment_features, periods= INSTALLMENTS_LAST_K_TREND_PERIODS)
    g = parallel_apply(gp, func, index_name='SK_ID_CURR', chunk_size=10000).reset_index()
    pay_agg = pay_agg.merge(g, on='SK_ID_CURR', how='left')

    # Last loan features
    g = parallel_apply(gp, installments_last_loan_features, index_name='SK_ID_CURR', chunk_size=10000).reset_index()
    pay_agg = pay_agg.merge(g, on='SK_ID_CURR', how='left')
    return pay_agg


def trend_in_last_k_instalment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'], ascending=False, inplace=True)
    features = {}

    for period in periods:
        gr_period = gr_.iloc[:period]
        features = add_trend_feature(features, gr_period, 'DPD',
                                           '{}_TREND_'.format(period))
        features = add_trend_feature(features, gr_period, 'PAID_OVER_AMOUNT',
                                           '{}_TREND_'.format(period))
    return features


def installments_last_loan_features(gr):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'], ascending=False, inplace=True)
    last_installment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV'] == last_installment_id]

    features = {}
    features = add_features_in_group(features, gr_, 'DPD',
                                     ['sum', 'mean', 'max', 'std'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'LATE_PAYMENT',
                                     ['count', 'mean'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'PAID_OVER_AMOUNT',
                                     ['sum', 'mean', 'max', 'min', 'std'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'PAID_OVER',
                                     ['count', 'mean'],
                                     'LAST_LOAN_')
    return features

# ------------------------- CREDIT CARD PIPELINE -------------------------

def get_credit_card(path, num_rows= None):
    """ Process credit_card_balance.csv and return a pandas dataframe. """
    cc = pd.read_csv(os.path.join(path, 'credit_card_balance.csv'), nrows= num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=False)
    cc.rename(columns={'AMT_RECIVABLE': 'AMT_RECEIVABLE'}, inplace=True)
    # Amount used from limit
    cc['LIMIT_USE'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
    # Current payment / Min payment
    cc['PAYMENT_DIV_MIN'] = cc['AMT_PAYMENT_CURRENT'] / cc['AMT_INST_MIN_REGULARITY']
    # Late payment
    cc['LATE_PAYMENT'] = cc['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    # How much drawing of limit
    cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
    # Aggregations by SK_ID_CURR
    cc_agg = cc.groupby('SK_ID_CURR').agg(CREDIT_CARD_AGG)
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    cc_agg.reset_index(inplace= True)

    # Last month balance of each credit card application
    last_ids = cc.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    last_months_df = cc[cc.index.isin(last_ids)]
    cc_agg = group_and_merge(last_months_df,cc_agg,'CC_LAST_', {'AMT_BALANCE': ['mean', 'max']})

    # Aggregations for last x months
    for months in [12, 24, 48]:
        cc_prev_id = cc[cc['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
        cc_recent = cc[cc['SK_ID_PREV'].isin(cc_prev_id)]
        prefix = 'INS_{}M_'.format(months)
        cc_agg = group_and_merge(cc_recent, cc_agg, prefix, CREDIT_CARD_TIME_AGG)
    return cc_agg

# Configurations

In [69]:
# just copied codes


# INSTALLMENTS TREND PERIODS
INSTALLMENTS_LAST_K_TREND_PERIODS =  [12, 24, 60, 120]

# LIGHTGBM CONFIGURATION AND HYPER-PARAMETERS
GENERATE_SUBMISSION_FILES = True
STRATIFIED_KFOLD = False
RANDOM_SEED = 737851
NUM_FOLDS = 10
EARLY_STOPPING = 100



LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',
    'n_estimators': 10000,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 1,
    'is_unbalance': False,
    'silent':-1,
    'verbose':-1
}
# AGGREGATIONS
BUREAU_AGG = {
    'SK_ID_BUREAU': ['nunique'],
    'DAYS_CREDIT': ['min', 'max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_ANNUITY': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    # Categorical
    'STATUS_0': ['mean'],
    'STATUS_1': ['mean'],
    'STATUS_12345': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed': ['mean'],
    'CREDIT_ACTIVE_Sold': ['mean'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],
    # Group by loan duration features (months)
    'LL_AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'LL_DEBT_CREDIT_DIFF': ['mean'],
    'LL_STATUS_12345': ['mean'],
}

BUREAU_ACTIVE_AGG = {
    'DAYS_CREDIT': ['max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['min', 'mean'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
}

BUREAU_CLOSED_AGG = {
    'DAYS_CREDIT': ['max', 'var'],
    'DAYS_CREDIT_ENDDATE': ['max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'sum'],
    'DAYS_CREDIT_UPDATE': ['max'],
    'ENDDATE_DIF': ['mean'],
    'STATUS_12345': ['mean'],
}

BUREAU_LOAN_TYPE_AGG = {
    'DAYS_CREDIT': ['mean', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DAYS_CREDIT_ENDDATE': ['max'],
}

BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'STATUS_0': ['mean'],
    'STATUS_12345': ['mean'],
}

PREVIOUS_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['max'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean', 'var'],
    'DOWN_PAYMENT_TO_CREDIT': ['mean'],
}

PREVIOUS_ACTIVE_AGG = {
    'SK_ID_PREV': ['nunique'],
    'SIMPLE_INTERESTS': ['mean'],
    'AMT_ANNUITY': ['max', 'sum'],
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['sum'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'AMT_PAYMENT': ['sum'],
    'INSTALMENT_PAYMENT_DIFF': ['mean', 'max'],
    'REMAINING_DEBT': ['max', 'mean', 'sum'],
    'REPAYMENT_RATIO': ['mean'],
}

PREVIOUS_APPROVED_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max'],
    'AMT_GOODS_PRICE': ['max'],
    'HOUR_APPR_PROCESS_START': ['min', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['mean'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['max'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    # The following features are only for approved applications
    'DAYS_FIRST_DRAWING': ['max', 'mean'],
    'DAYS_FIRST_DUE': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE': ['max', 'mean'],
    'DAYS_LAST_DUE_DIFF': ['min', 'max', 'mean'],
    'SIMPLE_INTERESTS': ['min', 'max', 'mean'],
}

PREVIOUS_REFUSED_AGG = {
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['min', 'max'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LATE_PAYMENTS_AGG = {
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LOAN_TYPE_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['min', 'mean', 'max', 'var'],
    'APPLICATION_CREDIT_DIFF': ['min', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['max'],
    'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean'],
    'CNT_PAYMENT': ['mean'],
}

PREVIOUS_TIME_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['mean', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

POS_CASH_AGG = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'SK_DPD': ['max', 'mean', 'sum', 'var'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
    'LATE_PAYMENT': ['mean']
}

INSTALLMENTS_AGG = {
    'SK_ID_PREV': ['size', 'nunique'],
    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],
    'LATE_PAYMENT': ['mean', 'sum'],
    'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
    'LATE_PAYMENT_RATIO': ['mean'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],
    'PAID_OVER': ['mean']
}

INSTALLMENTS_TIME_AGG = {
    'SK_ID_PREV': ['size'],
    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],
    'LATE_PAYMENT': ['mean'],
    'SIGNIFICANT_LATE_PAYMENT': ['mean'],
    'LATE_PAYMENT_RATIO': ['mean'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],
}

CREDIT_CARD_AGG = {
    'MONTHS_BALANCE': ['min'],
    'AMT_BALANCE': ['max'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['max'],
    'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
    'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
    'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'mean', 'sum', 'var'],
    'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
    'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_POS_CURRENT': ['mean'],
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['max', 'sum'],
    'LIMIT_USE': ['max', 'mean'],
    'PAYMENT_DIV_MIN': ['min', 'mean'],
    'LATE_PAYMENT': ['max', 'sum'],
}

CREDIT_CARD_TIME_AGG = {
    'CNT_DRAWINGS_ATM_CURRENT': ['mean'],
    'SK_DPD': ['max', 'sum'],
    'AMT_BALANCE': ['mean', 'max'],
    'LIMIT_USE': ['max', 'mean']
}

# Sequantial execution
풀어 씁니다.

In [70]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [71]:
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 100)

In [72]:
num_rows = 30000
debug = True


In [73]:
NUM_THREADS=4
DATA_DIRECTORY="../input/home-credit-default-risk/"
SUBMISSION_SUFIX="_model2_04"
path=DATA_DIRECTORY


In [74]:
with timer("application_train and application_test"):
    df = get_train_test(DATA_DIRECTORY, num_rows=num_rows)
    print("Application dataframe shape: ", df.shape)


Application dataframe shape:  (59999, 83)
application_train and application_test - done in 9s


__do_mean 등 utility 함수 훑어보기__
간단한 데이터프레임을 만들어 테스트 해봅시다. 이거는 group cols
```python
gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
    columns={counted: agg_name})
df = df.merge(gp, on=group_cols, how='left')
```

In [75]:
with timer("Bureau and bureau_balancce data"):
    bureau_df=get_bureau(DATA_DIRECTORY, num_rows=num_rows)
    df = pd.merge(df, bureau_df, on='SK_ID_CURR', how='left')
    print("Bureau dataframe shape: ", bureau_df.shape)
    del bureau_df; gc.collect()

Bureau dataframe shape:  (6076, 156)
Bureau and bureau_balancce data - done in 2s


In [76]:
with timer("previous_application"):
    prev_df = get_previous_applications(DATA_DIRECTORY, num_rows)
    df=pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
    del prev_df; gc.collect()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


previous_application - done in 3s


## GET POS CASH 함수 분석: sequential execution에서는 건너뛰어야 합니다.
```python
pos=get_pos_cash(DATA_DIRECTORY, num_rows)
df=pd.merge(df, pos, on='SK_ID_CURR', how='left')
```
이 부분부터 나눠서 뜯어봅시다.

In [38]:
pos = pd.read_csv(os.path.join(path, 'POS_CASH_balance.csv'), nrows=num_rows)
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


데이터 설명에서 이 파일에 대한 부분입니다만 ID부분을 제외하고 깊게 알아볼 필요는 없을 것 같습니다.
- SK_ID_PREV : 과거 Home Credit에서의 융자(credit) ID. 샘플에서 한 대출(loan)은 0~2개의 과거 융자(loan)와 연결될 수 있음 - loan과 credit을 크게 구별하지 않는 것 같습니다.
- SK_ID_CURR : 샘플에서 대출(loan)의 ID
- MONTHS_BALANCE : 대출 신청일과 balance(잔고?잔액?) 데이터가 몇개월이나 차이나는지. -1은 매달 확인하는 데이터의 자료라는 뜻이고, 0은 대출을 신청한 시점에서의 데이터라는 뜻입니다. 그런데 은행들이 Credit Bureau측에 업데이트를 자주 안해줘서 0이나 -1이나 의미 없을 수 있다고 하네요.
- CNT_INSTALMENT: Term of previous credit (can change over time)
- CNT_INSTALMENT_FUTURE: Installments left to pay on the previous credit
- NAME_CONTRACT_STATUS: Contract status during the month
- SK_DPD: 지난 융자에서 기한을 얼마나 넘겼는지.
- SK_DPD_DEF: PD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

In [39]:
#print(pos['SK_ID_PREV'].nunique(), pos['SK_ID_PREV'].value_counts())
print(pos['SK_ID_PREV'].nunique())
#print(pos.groupby('SK_ID_PREV')['SK_ID_CURR'].nunique().sum())
print(pos['SK_ID_CURR'].nunique())
#print(pos.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().sum())

28559
27135


In [40]:
pos.groupby('SK_ID_CURR')[['SK_ID_CURR', 'SK_ID_PREV']].nunique().sort_values(by='SK_ID_PREV').max()

SK_ID_CURR    1
SK_ID_PREV    4
dtype: int64

In [41]:
pos.groupby('SK_ID_PREV')[['SK_ID_CURR', 'SK_ID_PREV']].nunique().max()

SK_ID_CURR    1
SK_ID_PREV    1
dtype: int64

SK_ID_PREV, SK_ID_CURR 의 경우 description에서 설명한 대로 중복값이 꽤 있습니다. 그런데 어떤 식으로 중복값이 있을지 확인해 보았습니다. PREV->CURR 는 다대일 대응관계가 성립합니다.

In [42]:
#pos[pos['SK_ID_PREV'].duplicated(keep=False)].sort_values(by='SK_ID_PREV', axis=0).head()

In [43]:
pos, categorical_cols = one_hot_encoder(pos, nan_as_category= False)
categorical_cols

['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed']

만약 nrows를 제한하지 않는다면 NAME_CONTRACT_STATUS의 분포는 다음과 같습니다.  
```python
pos = pd.read_csv(os.path.join(path, 'POS_CASH_balance.csv'), nrows=None)
pos['NAME_CONTRACT_STATUS'].value_counts()
```
```python
Active                   9151119  
Completed                 744883  
Signed                     87260  
Demand                      7065  
Returned to the store       5461  
Approved                    4917  
Amortized debt               636  
Canceled                      15  
XNA                            2  
Name: NAME_CONTRACT_STATUS, dtype: int64  
```

In [45]:
#pos['NAME_CONTRACT_STATUS'].value_counts()

## Back to sequential execution

In [77]:
with timer("previous applications balance"):
    pos=get_pos_cash(DATA_DIRECTORY, num_rows)
    df=pd.merge(df, pos, on='SK_ID_CURR', how='left')
    print("Pos-cash dataframe shape: ", pos.shape)
    del pos; gc.collect()
    ins = get_installment_payments(DATA_DIRECTORY, num_rows)
    df=pd.merge(df, ins, on='SK_ID_CURR', how='left')
    print("Installments dataframe shape: ", ins.shape)
    del ins; gc.collect()
    cc=get_credit_card(DATA_DIRECTORY, num_rows)
    df=pd.merge(df, cc, on='SK_ID_CURR', how='left')
    print("Credit card dataframe shape: ", cc.shape)
    del cc; gc.collect()

Pos-cash dataframe shape:  (27135, 24)
Installments dataframe shape:  (22446, 101)
Credit card dataframe shape:  (24459, 59)
previous applications balance - done in 65s


In [51]:
#print([c for c in df.columns if 'BUREAU' in c])

In [78]:
#add ratios and groupby between different tables
df=add_ratios_features(df)

In [90]:

df=reduce_memory(df)
lgbm_categorical_feat=[
    'CODE_GENDER', 'FLAG_OWN_CAR','NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']
with timer("Run LightGBM"):
    feat_importance = kfold_lightgbm_sklearn(df, lgbm_categorical_feat)
    print(feat_importance)

Initial df memory usage is 109.12 MB for 659 columns
Final memory usage is: 109.12 MB - decreased by 0.0%
Train/valid shape: (29999, 659), test_shape: (30000, 659)


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future releas

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[400]	training's auc: 0.879428	training's binary_logloss: 0.211405	valid_1's auc: 0.737034	valid_1's binary_logloss: 0.238554
[800]	training's auc: 0.933191	training's binary_logloss: 0.181833	valid_1's auc: 0.74501	valid_1's binary_logloss: 0.235783
FOLD  1 AUC : 0.746129


...

[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


FOLD 10 AUC : 0.777957
Full AUC score 0.724187


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

                                    feature          gain  split
280                        EXT_SOURCES_MEAN  50420.774443  766.7
282                   EXT_SOURCES_NANMEDIAN  30954.682970  662.5
288                            EXT_SOURCE_3  16707.850373  971.7
281                         EXT_SOURCES_MIN  15867.782935  709.9
254                 CREDIT_TO_ANNUITY_RATIO  12062.662833  988.3
..                                      ...           ...    ...
94   BUREAU_CAR_AMT_CREDIT_MAX_OVERDUE_MEAN      0.000000    0.0
242                     CC_LATE_PAYMENT_SUM      0.000000    0.0
241                     CC_LATE_PAYMENT_MAX      0.000000    0.0
95       BUREAU_CAR_AMT_CREDIT_SUM_DEBT_MAX      0.000000    0.0
98           BUREAU_CAR_AMT_CREDIT_SUM_MEAN      0.000000    0.0

[657 rows x 3 columns]
Run LightGBM - done in 327s


In [86]:
[c for c in df.columns if "PREV" in c]

['PREV_SK_ID_PREV_NUNIQUE',
 'PREV_AMT_ANNUITY_MIN',
 'PREV_AMT_ANNUITY_MAX',
 'PREV_AMT_ANNUITY_MEAN',
 'PREV_AMT_DOWN_PAYMENT_MAX',
 'PREV_AMT_DOWN_PAYMENT_MEAN',
 'PREV_HOUR_APPR_PROCESS_START_MIN',
 'PREV_HOUR_APPR_PROCESS_START_MAX',
 'PREV_HOUR_APPR_PROCESS_START_MEAN',
 'PREV_RATE_DOWN_PAYMENT_MAX',
 'PREV_RATE_DOWN_PAYMENT_MEAN',
 'PREV_DAYS_DECISION_MIN',
 'PREV_DAYS_DECISION_MAX',
 'PREV_DAYS_DECISION_MEAN',
 'PREV_CNT_PAYMENT_MAX',
 'PREV_CNT_PAYMENT_MEAN',
 'PREV_DAYS_TERMINATION_MAX',
 'PREV_CREDIT_TO_ANNUITY_RATIO_MEAN',
 'PREV_CREDIT_TO_ANNUITY_RATIO_MAX',
 'PREV_APPLICATION_CREDIT_DIFF_MIN',
 'PREV_APPLICATION_CREDIT_DIFF_MAX',
 'PREV_APPLICATION_CREDIT_DIFF_MEAN',
 'PREV_APPLICATION_CREDIT_RATIO_MIN',
 'PREV_APPLICATION_CREDIT_RATIO_MAX',
 'PREV_APPLICATION_CREDIT_RATIO_MEAN',
 'PREV_APPLICATION_CREDIT_RATIO_VAR',
 'PREV_DOWN_PAYMENT_TO_CREDIT_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Approved_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Canceled_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Refu